In [ ]:
#Connect Google Drive

from google.colab import drive
drive.mount("/content/gdrive", force_remount = True)

Mounted at /content/gdrive


In [ ]:
#Import and install tools
!pip install pandasql
import os
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as sm
import numpy as np
import matplotlib.pyplot as plt
from sqlite3 import connect
from scipy.stats.mstats import winsorize, ks_2samp
import seaborn as sns
import pandasql as ps

conn = connect(':memory:', timeout = 10)
filePath = "/content/gdrive/MyDrive/FinancePaperFIMA"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Import CRSP
file = filePath + "/CRSPMonthly1990Through2022.csv"
crsp1990To2022 = pd.read_csv(file)
print(crsp1990To2022)
print(crsp1990To2022.columns)

<ipython-input-47-d35c1c9c03cc>:3: DtypeWarning: Columns (5,9,18,19,49,58) have mixed types. Specify dtype option on import or set low_memory=False.
  crsp1990To2022 = pd.read_csv(file)


         PERMNO      date  NAMEENDT  SHRCD  EXCHCD   SICCD    NCUSIP TICKER  \
0         10001  19900131       NaN   11.0     3.0    4920  39040610   GFGC   
1         10001  19900228       NaN   11.0     3.0    4920  39040610   GFGC   
2         10001  19900330       NaN   11.0     3.0    4920  39040610   GFGC   
3         10001  19900430       NaN   11.0     3.0    4920  39040610   GFGC   
4         10001  19900531       NaN   11.0     3.0    4920  39040610   GFGC   
...         ...       ...       ...    ...     ...     ...       ...    ...   
2975236   93436  20211130       NaN   11.0     3.0  9999.0  88160R10   TSLA   
2975237   93436  20211231       NaN   11.0     3.0  9999.0  88160R10   TSLA   
2975238   93436  20220131       NaN   11.0     3.0  9999.0  88160R10   TSLA   
2975239   93436  20220228       NaN   11.0     3.0  9999.0  88160R10   TSLA   
2975240   93436  20220331       NaN   11.0     3.0  9999.0  88160R10   TSLA   

                     COMNAM SHRCLS  ... CFACSHR    

In [ ]:
#Parse relevant variables
crsp1990To2022 = crsp1990To2022.filter(['TICKER', 'date', 'SHRCD', 'EXCHCD', 'SICCD', 'SHROUT', 'ALTPRC', 'RET', 'DLRET', 'DLSTCD'])

#Change column names
crsp1990To2022.columns = ['tic', 'date', 'shrcd', 'exchcd', 'siccd', 'shrout', 'altprc', 'ret', 'dlret', 'dlstcd']

#Convert variable types
crsp1990To2022['tic'] = crsp1990To2022['tic'].apply(str)
crsp1990To2022['date'] = pd.to_datetime(crsp1990To2022['date'], format = '%Y%m%d')
crsp1990To2022['shrcd'] = pd.to_numeric(crsp1990To2022['shrcd'], errors = 'coerce')
crsp1990To2022['exchcd'] = pd.to_numeric(crsp1990To2022['exchcd'], errors = 'coerce')
crsp1990To2022['siccd'] = pd.to_numeric(crsp1990To2022['siccd'], errors = 'coerce')
crsp1990To2022['shrout'] = pd.to_numeric(crsp1990To2022['shrout'], errors = 'coerce')
crsp1990To2022['altprc'] = pd.to_numeric(crsp1990To2022['altprc'], errors = 'coerce')
crsp1990To2022['ret'] = pd.to_numeric(crsp1990To2022['ret'], errors = 'coerce')
crsp1990To2022['dlret'] = pd.to_numeric(crsp1990To2022['dlret'], errors = 'coerce')
crsp1990To2022['dlstcd'] = pd.to_numeric(crsp1990To2022['dlstcd'], errors = 'coerce')

#Only keep dates after 2018
crsp2018To2022 = crsp1990To2022[crsp1990To2022['date'] >= "2018-01-01"]

#Only keep US based common stocks
crsp2018To2022 = crsp2018To2022[(crsp2018To2022['shrcd'] == 10) | (crsp2018To2022['shrcd'] == 11)]

#Only keep NYSE, AMEX, and NASDAQ
crsp2018To2022 = crsp2018To2022[(crsp2018To2022['exchcd'] == 1) | (crsp2018To2022['exchcd'] == 2) | (crsp2018To2022['exchcd'] == 3) | (crsp2018To2022['exchcd'] == 31) | (crsp2018To2022['exchcd'] == 32) | (crsp2018To2022['exchcd'] == 33)]

#convert SICCD to 2-digit level
crsp2018To2022['siccd'] = np.floor(crsp2018To2022['siccd'] / 100)

#Adjust delisting returns
crsp2018To2022['ret_adj'] = ""
for i in crsp2018To2022.index:
  if (pd.isnull(crsp2018To2022['dlstcd'][i])) or (crsp2018To2022['dlstcd'][i] == 100):
    crsp2018To2022['ret_adj'][i] = crsp2018To2022['ret'][i]
  elif(pd.notnull(crsp2018To2022['dlstcd'][i])) and (pd.notnull(crsp2018To2022['dlret'][i])):
    crsp2018To2022['ret_adj'][i] = crsp2018To2022['dlret'][i]
  elif(551 <= crsp2018To2022['dlstcd'][i] <= 574) or (crsp2018To2022['dlstcd'][i] in [500, 520, 580, 584]):
    crsp2018To2022['ret_adj'][i] = -0.3
  else:
    crsp2018To2022['ret_adj'][i] = -1

crsp2018To2022.drop('dlret', inplace = True, axis = 1)
crsp2018To2022.drop('dlstcd', inplace = True, axis = 1)

#Calculate market value
crsp2018To2022['market_value'] = np.abs(crsp2018To2022['altprc'] * crsp2018To2022['shrout'])

#Add year and month columns
crsp2018To2022['date'] = pd.to_datetime(crsp2018To2022['date'])
crsp2018To2022['year'] = crsp2018To2022['date'].dt.year
crsp2018To2022['month'] = crsp2018To2022['date'].dt.month

#Sort DF
crsp2018To2022 = crsp2018To2022.sort_values(by = ['tic', 'date'])

#Add deciles based on market value at the start of the year
g = crsp2018To2022.groupby(['year', 'tic'], as_index = False)['market_value'].first()
g['size_quantile'] = g.groupby(['year'])['market_value'].rank(pct = True)
g['size_decile'] = np.ceil(10 * g['size_quantile'])
g.drop(['market_value', 'size_quantile'], inplace = True, axis = 1)
crsp2018To2022 = crsp2018To2022.merge(g, on = ['year', 'tic'], how = 'left')

#Drop missing values
crsp2018To2022 = crsp2018To2022.dropna()

#Reindex
crsp2018To2022 = crsp2018To2022.reset_index()
crsp2018To2022.drop('index', inplace = True, axis = 1)

print(crsp2018To2022)

<ipython-input-48-5f0c442aa3eb>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp2018To2022['siccd'] = np.floor(crsp2018To2022['siccd'] / 100)
<ipython-input-48-5f0c442aa3eb>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp2018To2022['ret_adj'] = ""
<ipython-input-48-5f0c442aa3eb>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

         tic       date  shrcd  exchcd  siccd    shrout  altprc       ret  \
0          A 2018-01-31   11.0     1.0   38.0  323018.0   73.43  0.096461   
1          A 2018-02-28   11.0     1.0   38.0  322717.0   68.59 -0.065913   
2          A 2018-03-29   11.0     1.0   38.0  322477.0   66.90 -0.024639   
3          A 2018-04-30   11.0     1.0   38.0  322477.0   65.74 -0.015112   
4          A 2018-05-31   11.0     1.0   38.0  319952.0   61.92 -0.058108   
...      ...        ...    ...     ...    ...       ...     ...       ...   
191199  ZYXI 2021-12-31   11.0     3.0   99.0   39738.0    9.97 -0.221094   
191200  ZYXI 2022-01-31   11.0     3.0   99.0   43712.0    7.92 -0.116148   
191201  ZYXI 2022-01-31   11.0     3.0   99.0   43712.0    7.92 -0.116148   
191202  ZYXI 2022-02-28   11.0     3.0   99.0   43712.0    6.29 -0.205808   
191203  ZYXI 2022-03-31   11.0     3.0   99.0   39784.0    6.23 -0.009539   

         ret_adj  market_value  year  month  size_decile  
0       0.096461

In [ ]:
#Export compustat for customer supplier

tempCols = ['tic', 'datadate', 'sale', 'csho', 'prcc_c', 'prcc_f', 'conm', 'indfmt', 'datafmt']
file = filePath + "/CompustatAnnual1950to2022.csv"
tempDF = pd.read_csv(file, usecols = tempCols)[tempCols]
tempDF = tempDF[tempDF['datadate'] >= 20180101]
tempDF.to_csv("/content/gdrive/MyDrive/FinancePaperFIMA/compustat2018To2022.csv")

#Import Compustat CSV
file = filePath + "/CompustatAnnual1950to2022.csv"
columnsToUse = ['tic', 'datadate', 'fyear', 'indfmt', 'datafmt', 'che', 'act', 'lct', 'at', 'dp', 'ib', 'dlc', 'txp', 'oiadp']
compustat1950To2022 = pd.read_csv(file, usecols = columnsToUse)[columnsToUse]
print(compustat1950To2022)
print(compustat1950To2022.columns)


          tic  datadate   fyear indfmt datafmt       che        act  \
0        AE.2  19611231  1961.0   INDL     STD       NaN        NaN   
1        AE.2  19621231  1962.0   INDL     STD       NaN        NaN   
2        AE.2  19631231  1963.0   INDL     STD       NaN      0.408   
3        AE.2  19641231  1964.0   INDL     STD     0.269      0.718   
4        AE.2  19651231  1965.0   INDL     STD     0.031      0.725   
...       ...       ...     ...    ...     ...       ...        ...   
591305  IVCGF  20201231  2020.0   INDL     STD   558.000  12100.000   
591306  IVCGF  20211231  2021.0   INDL     STD  1020.181  12385.477   
591307  DTRUY  20191231  2019.0   INDL     STD  6532.226  31890.187   
591308  DTRUY  20201231  2020.0   INDL     STD  9012.671  30632.474   
591309  DTRUY  20211231  2021.0   INDL     STD  8358.207  31787.117   

              lct         at        dp        ib        dlc      txp     oiadp  
0             NaN        NaN       NaN     0.050        NaN      N

In [ ]:
#Clean compustat DF

#Change column names
compustat1950To2022.columns = ['tic', 'datadate', 'fyear', 'indfmt', 'datafmt', 'cash', 'ca', 'cl', 'ta', 'dep', 'ib', 'std', 'tp', 'oiadp']

#Convert each column to correct variable types
compustat1950To2022['tic'] = compustat1950To2022['tic'].apply(str) #firm identifier
compustat1950To2022['datadate'] = pd.to_datetime(compustat1950To2022['datadate'], format = '%Y%m%d') #date of data
compustat1950To2022['fyear'] = pd.to_numeric(compustat1950To2022['fyear'], errors = 'coerce') #fiscal year of data
compustat1950To2022['indfmt'] = compustat1950To2022['indfmt'].apply(str) #industry format
compustat1950To2022['datafmt'] = compustat1950To2022['datafmt'].apply(str) #data fmt
compustat1950To2022['cash'] = pd.to_numeric(compustat1950To2022['cash'], errors = 'coerce') #cash and short-term investements
compustat1950To2022['ca'] = pd.to_numeric(compustat1950To2022['ca'], errors = 'coerce') #current assets
compustat1950To2022['cl'] = pd.to_numeric(compustat1950To2022['cl'], errors = 'coerce') #current liabilities
compustat1950To2022['ta'] = pd.to_numeric(compustat1950To2022['ta'], errors = 'coerce') #total assets
compustat1950To2022['dep'] = pd.to_numeric(compustat1950To2022['dep'], errors = 'coerce') #depreciation and amortization
compustat1950To2022['ib'] = pd.to_numeric(compustat1950To2022['ib'], errors = 'coerce') #income before extraordinary items
compustat1950To2022['std'] = pd.to_numeric(compustat1950To2022['std'], errors = 'coerce') #debt in current liabilities
compustat1950To2022['tp'] = pd.to_numeric(compustat1950To2022['tp'], errors = 'coerce') #income taxes payable
compustat1950To2022['oiadp'] = pd.to_numeric(compustat1950To2022['oiadp'], errors = 'coerce') #operating income after depreciation

#Only keep data after 2017
compustat2018To2022 = compustat1950To2022[compustat1950To2022['datadate'] >= "2017-01-01"]

#Only keep correct formats
compustat2018To2022 = compustat2018To2022[compustat2018To2022['indfmt'] == "INDL"]
compustat2018To2022 = compustat2018To2022[compustat2018To2022['datafmt'] == "STD"]

#Drop infinite/null values
compustat2018To2022.replace([np.inf, -np.inf], np.nan, inplace = True)
compustat2018To2022 = compustat2018To2022.dropna()

#Reindex
compustat2018To2022 = compustat2018To2022.reset_index()
compustat2018To2022.drop('index', inplace = True, axis = 1)

#Calculate change in variables
compustat2018To2022['delta_cash'] = compustat2018To2022.groupby(['tic'])['cash'].diff()
compustat2018To2022['delta_ca'] = compustat2018To2022.groupby(['tic'])['ca'].diff()
compustat2018To2022['delta_cl'] = compustat2018To2022.groupby(['tic'])['cl'].diff()
compustat2018To2022['delta_std'] = compustat2018To2022.groupby(['tic'])['std'].diff()
compustat2018To2022['delta_tp'] = compustat2018To2022.groupby(['tic'])['tp'].diff()

#Drop null values
compustat2018To2022 = compustat2018To2022.dropna()

#Calculate accruals
compustat2018To2022['accruals'] = (compustat2018To2022['delta_ca'] - compustat2018To2022['delta_cash']) - (compustat2018To2022['delta_cl'] - compustat2018To2022['delta_std'] - compustat2018To2022['delta_tp']) - compustat2018To2022['dep']
compustat2018To2022['accruals_subcomp_1'] = compustat2018To2022['delta_ca'] - compustat2018To2022['delta_cash']
compustat2018To2022['accruals_subcomp_2'] = -(compustat2018To2022['delta_cl'] - compustat2018To2022['delta_std'] - compustat2018To2022['delta_tp'])
compustat2018To2022['accruals_subcomp_3'] = -(compustat2018To2022['dep'])

#Calculate total assets
avgATDF = compustat2018To2022.groupby(['tic'], as_index = False)['ta'].mean()
avgATDF.columns = ['tic', 'avg_at']
compustat2018To2022 = compustat2018To2022.merge(avgATDF, on = 'tic', how = 'left')

#Calculate earnings component
compustat2018To2022['earnings_comp'] = compustat2018To2022['oiadp'] / compustat2018To2022['avg_at']

#Calculate accruals component
compustat2018To2022['accruals_comp'] = compustat2018To2022['accruals'] / compustat2018To2022['avg_at']

#Calculate cash flow component
compustat2018To2022['cash_flows_comp'] = compustat2018To2022['earnings_comp'] / compustat2018To2022['accruals_comp']

#Standardize sub-components
compustat2018To2022['accruals_subcomp_1'] /= compustat2018To2022['avg_at']
compustat2018To2022['accruals_subcomp_2'] /= compustat2018To2022['avg_at']
compustat2018To2022['accruals_subcomp_3'] /= compustat2018To2022['avg_at']
#Drop infinite/null values
compustat2018To2022.replace([np.inf, -np.inf], np.nan, inplace = True)
compustat2018To2022 = compustat2018To2022.dropna()
#Reindex
compustat2018To2022 = compustat2018To2022.reset_index()
compustat2018To2022.drop(['index'], inplace = True, axis = 1)
print(compustat2018To2022)

         tic   datadate   fyear indfmt datafmt      cash         ca  \
0        AIR 2018-05-31  2017.0   INDL     STD    41.600    942.700   
1        AIR 2019-05-31  2018.0   INDL     STD    41.100    952.500   
2        AIR 2020-05-31  2019.0   INDL     STD   424.700   1438.700   
3        AIR 2021-05-31  2020.0   INDL     STD    60.200    937.000   
4        AIR 2022-05-31  2021.0   INDL     STD    58.900   1007.200   
...      ...        ...     ...    ...     ...       ...        ...   
24301   KARO 2021-02-28  2020.0   INDL     STD    65.057     87.432   
24302   KARO 2022-02-28  2021.0   INDL     STD    47.427     72.275   
24303  IVCGF 2020-12-31  2020.0   INDL     STD   558.000  12100.000   
24304  DTRUY 2020-12-31  2020.0   INDL     STD  9012.671  30632.474   
24305  DTRUY 2021-12-31  2021.0   INDL     STD  8358.207  31787.117   

              cl         ta       dep  ...  delta_std  delta_tp  accruals  \
0        333.300   1524.700    40.500  ...     -2.000   -12.300   -30.

In [ ]:
#Generate mainDF with merged CRSP and Compustat
crsp2018To2022.to_sql("crsp2018To2022", conn, if_exists = 'replace')
compustat2018To2022.to_sql("compustat2018To2022", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT crsp2018To2022.*, compustat2018To2022.earnings_comp, compustat2018To2022.accruals_comp, compustat2018To2022.cash_flows_comp,
        compustat2018To2022.accruals_subcomp_1, compustat2018To2022.accruals_subcomp_2, compustat2018To2022.accruals_subcomp_3
        FROM crsp2018To2022
        LEFT JOIN compustat2018To2022
        ON crsp2018To2022.tic = compustat2018To2022.tic
        AND crsp2018To2022.date = compustat2018To2022.datadate
        '''

mainDF = pd.read_sql(query, conn)
mainDF.drop('index', inplace = True, axis = 1)
print(mainDF)

         tic                 date  shrcd  exchcd  siccd    shrout  altprc  \
0          A  2018-01-31 00:00:00   11.0     1.0   38.0  323018.0   73.43   
1          A  2018-02-28 00:00:00   11.0     1.0   38.0  322717.0   68.59   
2          A  2018-03-29 00:00:00   11.0     1.0   38.0  322477.0   66.90   
3          A  2018-04-30 00:00:00   11.0     1.0   38.0  322477.0   65.74   
4          A  2018-05-31 00:00:00   11.0     1.0   38.0  319952.0   61.92   
...      ...                  ...    ...     ...    ...       ...     ...   
191199  ZYXI  2021-12-31 00:00:00   11.0     3.0   99.0   39738.0    9.97   
191200  ZYXI  2022-01-31 00:00:00   11.0     3.0   99.0   43712.0    7.92   
191201  ZYXI  2022-01-31 00:00:00   11.0     3.0   99.0   43712.0    7.92   
191202  ZYXI  2022-02-28 00:00:00   11.0     3.0   99.0   43712.0    6.29   
191203  ZYXI  2022-03-31 00:00:00   11.0     3.0   99.0   39784.0    6.23   

             ret    ret_adj  market_value  year  month  size_decile  \
0   

In [ ]:
#Import Fama French

file = filePath + "/F-F_Research_Data_5_Factors.csv"
ffDF = pd.read_csv(file)
print(ffDF)
print(ffDF.columns)

       Date  Mkt_RF   SMB    HML   RMW   CMA    RF
0    196307   -0.39 -0.44  -0.89  0.68 -1.23  0.27
1    196308    5.07 -0.75   1.68  0.36 -0.34  0.25
2    196309   -1.57 -0.55   0.08 -0.71  0.29  0.27
3    196310    2.53 -1.37  -0.14  2.80 -2.02  0.29
4    196311   -0.85 -0.89   1.81 -0.51  2.31  0.27
..      ...     ...   ...    ...   ...   ...   ...
702  202201   -6.25 -3.95  12.74  0.73  7.73  0.00
703  202202   -2.29  2.90   3.09 -2.12  2.99  0.00
704  202203    3.06 -2.14  -1.82 -1.32  3.24  0.00
705  202204   -9.45 -0.38   6.16  3.51  5.87  0.00
706  202205   -0.34  0.02   8.38  1.61  3.82  0.03

[707 rows x 7 columns]
Index(['Date', 'Mkt_RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF'], dtype='object')


In [ ]:
#Clean FF Dataframe

#Change columns names
ffDF.columns = ['date', 'rm_minus_rf', 'smb', 'hml', 'rmw', 'cma', 'rf']

#Add month and year columns
ffDF['year'] = ""
ffDF['month'] = ""
for i in ffDF.index:
  ffDF['year'][i] = int(str(ffDF['date'][i])[0:4]) #take first 4 digits of date (year)
  ffDF['month'][i] = int(str(ffDF['date'][i])[-2] + str(ffDF['date'][i])[-1]) #take last two digits of date (month)

#Change values to decimal
ffDF['rm_minus_rf'] /= 100
ffDF['smb'] /= 100
ffDF['hml'] /= 100
ffDF['rmw'] /= 100
ffDF['cma'] /= 100
ffDF['rf'] /= 100

#Only keep dates after 2018
ffDF = ffDF[ffDF['date'] >= 201801]

#Reindex
ffDF = ffDF.reset_index()
ffDF.drop('index', inplace = True, axis = 1)

#Calculate annual market return - risk free rate
annualRmMinusRf = pd.DataFrame(index = [2018, 2019, 2020, 2021], columns = ['year', 'annual_rm_minus_rf'])
annualRmMinusRf['year'] = annualRmMinusRf.index
tempDF = ffDF.loc[0:11]
tempDF['rm_minus_rf_compounded'] = (1 + tempDF['rm_minus_rf']).cumprod() - 1
annualRmMinusRf['annual_rm_minus_rf'][2018] = tempDF['rm_minus_rf_compounded'][11]
tempDF = ffDF.loc[12:23]
tempDF['rm_minus_rf_compounded'] = (1 + tempDF['rm_minus_rf']).cumprod() - 1
annualRmMinusRf['annual_rm_minus_rf'][2019] = tempDF['rm_minus_rf_compounded'][23]
tempDF = ffDF.loc[24:35]
tempDF['rm_minus_rf_compounded'] = (1 + tempDF['rm_minus_rf']).cumprod() - 1
annualRmMinusRf['annual_rm_minus_rf'][2020] = tempDF['rm_minus_rf_compounded'][35]
tempDF = ffDF.loc[36:47]
tempDF['rm_minus_rf_compounded'] = (1 + tempDF['rm_minus_rf']).cumprod() - 1
annualRmMinusRf['annual_rm_minus_rf'][2021] = tempDF['rm_minus_rf_compounded'][47]

#Left join relevant FF (risk-free rate) data into main DF
mainDF.to_sql("mainDF", conn, if_exists = 'replace')
ffDF.to_sql("ffDF", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT mainDF.*, ffDF.rf, ffDF.rm_minus_rf
        FROM mainDF
        LEFT JOIN ffDF
        ON mainDF.year = ffDF.year
        AND mainDF.month = ffDF.month
        '''
mainDF = pd.read_sql(query, conn)
mainDF['date'] = pd.to_datetime(mainDF['date'])
mainDF['ret'] = pd.to_numeric(mainDF['ret'], errors = 'coerce')
mainDF['ret_adj'] = pd.to_numeric(mainDF['ret_adj'], errors = 'coerce')
mainDF['size_decile'] = pd.to_numeric(mainDF['size_decile'], errors = 'coerce')
mainDF['ret_minus_rf'] = mainDF['ret_adj'] - mainDF['rf']

#Add abnormal return data
g = mainDF.groupby(['year', 'month', 'size_decile'], as_index = False).apply(lambda x: np.average(x['ret_adj']))
g.columns = ['year', 'month', 'size_decile', 'avg_ret_on_size_decile']
mainDF = mainDF.merge(g, on = ['year', 'month', 'size_decile'], how = 'left')
mainDF['abn_ret'] = mainDF['ret_adj'] - mainDF['avg_ret_on_size_decile']
mainDF.drop('index', inplace = True, axis = 1)
mainDF.reset_index()
mainDF.dropna()
#Store mainDF
tempMain = mainDF

print(mainDF)


<ipython-input-53-237a12b801d3>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ffDF['year'][i] = int(str(ffDF['date'][i])[0:4]) #take first 4 digits of date (year)
<ipython-input-53-237a12b801d3>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ffDF['month'][i] = int(str(ffDF['date'][i])[-2] + str(ffDF['date'][i])[-1]) #take last two digits of date (month)
<ipython-input-53-237a12b801d3>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

         tic       date  shrcd  exchcd  siccd    shrout  altprc       ret  \
0          A 2018-01-31   11.0     1.0   38.0  323018.0   73.43  0.096461   
1          A 2018-02-28   11.0     1.0   38.0  322717.0   68.59 -0.065913   
2          A 2018-03-29   11.0     1.0   38.0  322477.0   66.90 -0.024639   
3          A 2018-04-30   11.0     1.0   38.0  322477.0   65.74 -0.015112   
4          A 2018-05-31   11.0     1.0   38.0  319952.0   61.92 -0.058108   
...      ...        ...    ...     ...    ...       ...     ...       ...   
191199  ZYXI 2021-12-31   11.0     3.0   99.0   39738.0    9.97 -0.221094   
191200  ZYXI 2022-01-31   11.0     3.0   99.0   43712.0    7.92 -0.116148   
191201  ZYXI 2022-01-31   11.0     3.0   99.0   43712.0    7.92 -0.116148   
191202  ZYXI 2022-02-28   11.0     3.0   99.0   43712.0    6.29 -0.205808   
191203  ZYXI 2022-03-31   11.0     3.0   99.0   39784.0    6.23 -0.009539   

         ret_adj  market_value  ...  accruals_comp  cash_flows_comp  \
0   

In [ ]:
#Replicate Table 1A

mainDF = tempMain

#Generate DF with non-null components (end of fiscal year data)
mainOnFYearEnd = mainDF.dropna()
mainOnFYearEnd['accruals_comp_quantile'] = mainOnFYearEnd['accruals_comp'].rank(pct = True)
mainOnFYearEnd['accruals_comp_decile'] = np.ceil(mainOnFYearEnd['accruals_comp_quantile'] * 10)

#Generate data for firms with December fiscal-year ends (for CAPM regressions)
mainOnFYearEndDecOnly = mainOnFYearEnd[mainOnFYearEnd['month'] == 12]

#Generate summary statistics (Table 1A)
table1A = pd.DataFrame(index = ['accruals_mean', 'accruals_median', 'cash_flows_mean', 'cash_flows_median', 'earnings_mean', 'earnings_median'], columns = range(1, 11))
for i in table1A.columns:
  tempDF = mainOnFYearEnd[mainOnFYearEnd['accruals_comp_decile'] == i]
  table1A[i]['accruals_mean'] = tempDF['accruals_comp'].mean()
  table1A[i]['accruals_median'] = tempDF['accruals_comp'].median()
  table1A[i]['cash_flows_mean'] = tempDF['cash_flows_comp'].mean()
  table1A[i]['cash_flows_median'] = tempDF['cash_flows_comp'].median()
  table1A[i]['earnings_mean'] = tempDF['earnings_comp'].mean()
  table1A[i]['earnings_median'] = tempDF['earnings_comp'].median()

print(table1A)

                         1         2         3         4         5         6   \
accruals_mean     -0.189894 -0.089031 -0.063193 -0.047544 -0.035427 -0.024555   
accruals_median   -0.147419 -0.088124 -0.062777 -0.047807 -0.035563 -0.024597   
cash_flows_mean     1.08581  0.849031  0.433792  0.341183  0.875631  1.392742   
cash_flows_median  0.332357 -0.252192 -0.632714 -1.019276 -1.349501 -1.995764   
earnings_mean     -0.219157  -0.07587 -0.027436  -0.01622 -0.030996 -0.032371   
earnings_median   -0.056139  0.022564  0.040335  0.048434  0.047548  0.047679   

                         7                     8         9         10  
accruals_mean      -0.01346              -0.00034  0.021679  0.142687  
accruals_median   -0.013717             -0.000742  0.020226  0.094279  
cash_flows_mean    3.395969  2036100260239.764404 -7.067792 -2.116516  
cash_flows_median -2.766586             -0.525362 -0.461314 -0.542677  
earnings_mean     -0.040971             -0.075646  -0.12773 -0.233787  


<ipython-input-54-56cd44d41d91>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mainOnFYearEnd['accruals_comp_quantile'] = mainOnFYearEnd['accruals_comp'].rank(pct = True)
<ipython-input-54-56cd44d41d91>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mainOnFYearEnd['accruals_comp_decile'] = np.ceil(mainOnFYearEnd['accruals_comp_quantile'] * 10)


In [ ]:
#Replicate Table 1B

mainDF = tempMain

#Initialize empty DataFrames
table1B = pd.DataFrame(columns = range(1, 11))
annualRetMinusRf = pd.DataFrame(index = range(2018,2022), columns = range(1, 11))
for i in annualRetMinusRf.index:
  for j in annualRetMinusRf.columns:
    annualRetMinusRf[j][i] = []

sizeOfPortfolios = pd.DataFrame(index = ['size', 'size_mean', 'size_median'], columns = range(1, 11))
for i in sizeOfPortfolios.index:
  for j in sizeOfPortfolios.columns:
    sizeOfPortfolios[j][i] = []

#Calculate compound ret - rf
for i in table1B.columns:
  tempDF = mainOnFYearEndDecOnly[mainOnFYearEndDecOnly['accruals_comp_decile'] == i]
  for j in tempDF.index:
    if((j - 11) in mainDF.index) and (mainDF['tic'][j - 11] == tempDF['tic'][j]): #check if index exists and is the same firm as current data
      tempDF2 = mainDF.loc[(j - 11) : j]
      tempDF2['ret_minus_rf_compounded'] = (1 + tempDF2['ret_minus_rf']).cumprod() - 1
      yearOfData = tempDF2['year'][j]
      annualRetMinusRf[i][yearOfData].append(tempDF2['ret_minus_rf_compounded'][j]) #add last count of compounded data to annual ret minus Rf data
      sizeOfPortfolios[i]['size'].append(np.log(tempDF2['market_value'][j]))

#Take average for each portfolio
for i in annualRetMinusRf.index:
  for j in annualRetMinusRf.columns:
    annualRetMinusRf[j][i] = np.average(annualRetMinusRf[j][i])

for j in sizeOfPortfolios.columns:
  sizeOfPortfolios[j]['size_mean'] = np.average(sizeOfPortfolios[j]['size'])
  sizeOfPortfolios[j]['size_median'] = np.median(sizeOfPortfolios[j]['size'])

sizeOfPortfolios.drop('size', inplace = True, axis = 0)
annualRetMinusRf['year'] = annualRetMinusRf.index
print(sizeOfPortfolios)


In [ ]:
#Generate DataFrame for Table 1B Regressions
dataForTable1BReg = annualRetMinusRf.merge(annualRmMinusRf, on = 'year', how = 'left')
dataForTable1BReg = dataForTable1BReg.astype(float)

#Run regressions to get alpha and beta
for i in range(1, 11):
  tempDF = dataForTable1BReg.filter([i, 'annual_rm_minus_rf'])
  tempDF.columns = ['current_decile_data', 'annual_rm_minus_rf']
  tempReg = sm.ols(formula = "current_decile_data ~ annual_rm_minus_rf", data = tempDF).fit()
  tempRegSummary = tempReg.summary()
  print("Accruals decile = " + str(i) + ":")
  print(tempRegSummary)

In [ ]:
#Replicate Table 1C

mainDF = tempMain

#Generate DF for Table 1C
table1C = pd.DataFrame(index = ['current_asset_mean', 'current_asset_median', 'current_liability_mean', 'current_liability_median', 'depreciation_expense_mean', 'deprecation_expense_median'], columns = range(1,11))
for i in table1C.columns:
  tempDF = mainOnFYearEnd[mainOnFYearEnd['accruals_comp_decile'] == i]
  table1C[i]['current_asset_mean'] = tempDF['accruals_subcomp_1'].mean()
  table1C[i]['current_asset_median'] = tempDF['accruals_subcomp_1'].median()
  table1C[i]['current_liability_mean'] = tempDF['accruals_subcomp_2'].mean()
  table1C[i]['current_liability_median'] = tempDF['accruals_subcomp_2'].median()
  table1C[i]['depreciation_expense_mean'] = tempDF['accruals_subcomp_3'].mean()
  table1C[i]['depreciation_expense_median'] = tempDF['accruals_subcomp_3'].median()
print(table1C)

In [ ]:
#Replicate Table 2

#Regenerate mainOnFYearEnd (to allow us to rerun the same code block for debugging)
mainDF = tempMain
mainOnFYearEnd = mainDF.dropna()
mainOnFYearEnd['accruals_comp_quantile'] = mainOnFYearEnd['accruals_comp'].rank(pct = True)
mainOnFYearEnd['accruals_comp_decile'] = np.ceil(mainOnFYearEnd['accruals_comp_quantile'] * 10)
mainOnFYearEnd['earnings_comp_quantile'] = mainOnFYearEnd['earnings_comp'].rank(pct = True)
mainOnFYearEnd['earnings_comp_decile'] = np.ceil(mainOnFYearEnd['earnings_comp_quantile'] * 10)
mainOnFYearEnd['cash_flows_comp_quantile'] = mainOnFYearEnd['cash_flows_comp'].rank(pct = True)
mainOnFYearEnd['cash_flows_comp_decile'] = np.ceil(mainOnFYearEnd['cash_flows_comp_quantile'] * 10)

#Generate column for next year's earnings component and accruals decile
mainOnFYearEnd['accruals_comp_next_year'] = mainOnFYearEnd.groupby(['tic'])['accruals_comp'].shift(-1)
mainOnFYearEnd['earnings_comp_next_year'] = mainOnFYearEnd.groupby(['tic'])['earnings_comp'].shift(-1)
mainOnFYearEnd['cash_flows_comp_next_year'] = mainOnFYearEnd.groupby(['tic'])['cash_flows_comp'].shift(-1)
mainOnFYearEnd['accruals_comp_decile_next_year'] = mainOnFYearEnd.groupby(['tic'])['accruals_comp_decile'].shift(-1)
mainOnFYearEnd['earnings_comp_decile_next_year'] = mainOnFYearEnd.groupby(['tic'])['earnings_comp_decile'].shift(-1)
mainOnFYearEnd['cash_flows_comp_decile_next_year'] = mainOnFYearEnd.groupby(['tic'])['cash_flows_comp_decile'].shift(-1)

#Generate DF for table 2 regressions (drops most recent year: no available next-year data)
dataForTable2Reg = mainOnFYearEnd.dropna()

#Run regressions
table2AReg = sm.ols(formula = "earnings_comp_next_year ~ earnings_comp", data = dataForTable2Reg).fit()
table2ARegSummary = table2AReg.summary()
table2BReg = sm.ols(formula = "earnings_comp_decile_next_year ~ earnings_comp_decile", data = dataForTable2Reg).fit()
table2BRegSummary = table2BReg.summary()

#Run regressions for firms grouped by industry codes
alpha0DistTable2A = []
alpha0DistTable2B = []
alpha1DistTable2A = []
alpha1DistTable2B = []

for i in dataForTable2Reg['siccd'].unique():
  tempDF = dataForTable2Reg[dataForTable2Reg['siccd'] == i]
  regA = sm.ols(formula = "earnings_comp_next_year ~ earnings_comp", data = tempDF).fit()
  alpha0DistTable2A.append(regA.params[0])
  alpha1DistTable2A.append(regA.params[1])
  regB = sm.ols(formula = "earnings_comp_decile_next_year ~ earnings_comp_decile", data = tempDF).fit()
  alpha0DistTable2B.append(regB.params[0])
  alpha1DistTable2B.append(regB.params[1])

#Print Table 2A results
print("Table 2A")
print("Pooled Regressions")
print(table2ARegSummary)
print("Industry Level Regressions")
print("Mean - alpha_0: " + str(np.average(alpha0DistTable2A)) + " alpha_1: " + str(np.average(alpha1DistTable2A)))
print("Q1 - alpha_0: " + str(np.quantile(alpha0DistTable2A, 0.25)) + " alpha_1: " + str(np.quantile(alpha1DistTable2A, 0.25)))
print("Median - alpha_0: " + str(np.median(alpha0DistTable2A)) + " alpha_1: " + str(np.median(alpha1DistTable2A)))
print("Q3 - alpha_0: " + str(np.quantile(alpha0DistTable2A, 0.75)) + " alpha_1: " + str(np.quantile(alpha1DistTable2A, 0.75)))

#Print Table 2B Results
print("Table 2B")
print("Pooled Regressions")
print(table2BRegSummary)
print("Industry Level Regressions")
print("Mean - alpha_0: " + str(np.average(alpha0DistTable2B)) + " alpha_1: " + str(np.average(alpha1DistTable2B)))
print("Q1 - alpha_0: " + str(np.quantile(alpha0DistTable2B, 0.25)) + " alpha_1: " + str(np.quantile(alpha1DistTable2B, 0.25)))
print("Median - alpha_0: " + str(np.median(alpha0DistTable2B)) + " alpha_1: " + str(np.median(alpha1DistTable2B)))
print("Q3 - alpha_0: " + str(np.quantile(alpha0DistTable2B, 0.75)) + " alpha_1: " + str(np.quantile(alpha1DistTable2B, 0.75)))

In [ ]:
#Store mainOnFYearEnd for future use
tempMainOnFYearEnd = mainOnFYearEnd

#Replicate Table 3
dataForTable3Reg = dataForTable2Reg

#Run Regressions
table3AReg = sm.ols(formula = "earnings_comp_next_year ~ accruals_comp + cash_flows_comp", data = dataForTable3Reg).fit()
table3ARegSummary = table3AReg.summary()
table3BReg = sm.ols(formula ="earnings_comp_decile_next_year ~ accruals_comp_decile + cash_flows_comp_decile", data = dataForTable3Reg).fit()
table3BRegSummary = table3BReg.summary()

#Run regressions for firms grouped by industry codes
gamma0DistTable3A = []
gamma1DistTable3A = []
gamma2DistTable3A = []
gamma0DistTable3B = []
gamma1DistTable3B = []
gamma2DistTable3B = []

for i in dataForTable3Reg['siccd'].unique():
  tempDF = dataForTable3Reg[dataForTable3Reg['siccd'] == i]
  regA = sm.ols(formula = "earnings_comp_next_year ~ accruals_comp + cash_flows_comp", data = tempDF).fit()
  gamma0DistTable3A.append(regA.params[0])
  gamma1DistTable3A.append(regA.params[1])
  gamma2DistTable3A.append(regA.params[2])
  regB = sm.ols(formula = "earnings_comp_decile_next_year ~ accruals_comp_decile + cash_flows_comp_decile", data = tempDF).fit()
  gamma0DistTable3B.append(regB.params[0])
  gamma1DistTable3B.append(regB.params[1])
  gamma2DistTable3B.append(regB.params[2])

#Remove min and max values
gamma0DistTable3A.sort()
gamma0DistTable3A = gamma0DistTable3A[1 : -1]
gamma1DistTable3A.sort()
gamma1DistTable3A = gamma1DistTable3A[1 : -1]
gamma2DistTable3A.sort()
gamma2DistTable3A = gamma2DistTable3A[1 : -1]
gamma0DistTable3B.sort()
gamma0DistTable3B = gamma0DistTable3B[1 : -1]
gamma1DistTable3B.sort()
gamma1DistTable3B = gamma1DistTable3B[1 : -1]
gamma2DistTable3B.sort()
gamma2DistTable3B = gamma2DistTable3B[1 : -1]

#Print Table 3A results
print("Table 3A")
print("Pooled Regressions")
print(table3ARegSummary)
print("Industry Level Regressions")
print("Mean - gamma_0: " + str(np.average(gamma0DistTable3A)) + " gamma_1: " + str(np.average(gamma1DistTable3A)) + " gamma_2: " + str(np.average(gamma2DistTable3A)))
print("Q1 - gamma_0: " + str(np.quantile(gamma0DistTable3A, 0.25)) + " gamma_1: " + str(np.quantile(gamma1DistTable3A, 0.25)) + " gamma_2: " + str(np.quantile(gamma2DistTable3A, 0.25)))
print("Median - gamma_0: " + str(np.median(gamma0DistTable3A)) + " gamma_1: " + str(np.median(gamma1DistTable3A)) + " gamma_2: " + str(np.median(gamma2DistTable3A)))
print("Q3 - gamma_0: " + str(np.quantile(gamma0DistTable3A, 0.75)) + " gamma_1: " + str(np.quantile(gamma1DistTable3A, 0.75)) + " gamma_2: " + str(np.quantile(gamma2DistTable3A, 0.75)))

#Print Table 3B results
print("Table 3B")
print("Pooled Regressions")
print(table3BRegSummary)
print("Industry Level Regressions")
print("Mean - gamma_0: " + str(np.average(gamma0DistTable3B)) + " gamma_1: " + str(np.average(gamma1DistTable3B)) + " gamma_2: " + str(np.average(gamma2DistTable3B)))
print("Q1 - gamma_0: " + str(np.quantile(gamma0DistTable3B, 0.25)) + " gamma_1: " + str(np.quantile(gamma1DistTable3B, 0.25)) + " gamma_2: " + str(np.quantile(gamma2DistTable3B, 0.255)))
print("Median - gamma_0: " + str(np.median(gamma0DistTable3B)) + " gamma_1: " + str(np.median(gamma1DistTable3B)) + " gamma_2: " + str(np.median(gamma2DistTable3B)))
print("Q3 - gamma_0: " + str(np.quantile(gamma0DistTable3B, 0.75)) + " gamma_1: " + str(np.quantile(gamma1DistTable3B, 0.75)) + " gamma_2: " + str(np.quantile(gamma2DistTable3B, 0.75)))